## Libs

In [1]:
import matplotlib as matpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math

# Using the Tensorflow backend (default).
from keras.models import Sequential
from keras.layers.recurrent import LSTM
from keras.layers.core import Dense, Activation, Dropout
from tensorflow import set_random_seed
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.utils import shuffle
from keras.callbacks import EarlyStopping

# advanced plotting
import seaborn as sns
plt.style.use('seaborn-darkgrid')
%matplotlib inline

Using TensorFlow backend.


## Data

In [17]:
%store -r data_IVV

In [18]:
mm = MinMaxScaler(feature_range = (0,1))
dataset = mm.fit_transform(data_IVV)

In [19]:
split = 0.6

train_size = int(len(dataset) * split)
#validation
test_size = len(dataset) - train_size

train, test = dataset[0:train_size, :], dataset[train_size:len(dataset), :]

print("training, test set: " + str((len(train), len(test))))

training, test set: (313, 209)


In [20]:
def input_dataset(dataset, window):
    data_X, data_y = [], []
    for i in range(len(dataset) - window - 1):
        a = dataset[i:(i + window), 0]
        data_X.append(a)
        data_y.append(dataset[i + window, 0])
    return(np.array(data_X), np.array(data_y))


In [21]:
# New testing and training sets for rolling forecast.
window = 1
train_X, train_Y = input_dataset(train, window)
test_X, test_Y = input_dataset(test, window)
print("Original train shape:")
print(train_X.shape)

# Reshape input data to match Keras format.
train_X = np.reshape(train_X, (train_X.shape[0], 1, train_X.shape[1]))
test_X = np.reshape(test_X, (test_X.shape[0], 1, test_X.shape[1]))
print("New train shape:")
print(train_X.shape)

Original train shape:
(311, 1)
New train shape:
(311, 1, 1)


## 1 Epoch

In [7]:
def fit_LSTM(train_X, train_Y, window = 1, neurons=128):
    set_random_seed(3)
    model = Sequential()
    
    model.add(LSTM(neurons, 
                   input_shape = (1, window)
                  ))
    model.add(Dense(1))
    model.compile(loss = 'mean_squared_error', 
                  optimizer = 'adam')
    earlyStop=EarlyStopping(monitor='val_loss',verbose=2,patience=15)
    model.fit(train_X, 
              train_Y, 
              epochs = 1, 
              batch_size = 1,
              shuffle = False
             # verbose = 2
             )
    
    return(model)

# Fit the first model.
model1 = fit_LSTM(train_X, train_Y, window)

W0918 23:41:42.644801 4545566144 deprecation_wrapper.py:119] From //anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0918 23:41:42.646423 4545566144 deprecation_wrapper.py:119] From //anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0918 23:41:42.652539 4545566144 deprecation_wrapper.py:119] From //anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0918 23:41:42.848624 4545566144 deprecation_wrapper.py:119] From //anaconda3/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0918 23:41:43.082672 4545566144 deprecation.py:323] From //anaconda3/li

Epoch 1/1
311/311 [==============================] - 4s 11ms/step - loss: 3.7311e-04


In [8]:
def prediction_score(model, X, Y):
    # Make predictions using input data
    pred = mm.inverse_transform(model.predict(X))
    # Show Y on original scale
    original_data = mm.inverse_transform([Y])
    # RMSE.
    score = math.sqrt(mean_squared_error(original_data[0], pred[:, 0]))
    return(score, pred)

In [9]:
rmse_test, test_pred = prediction_score(model1, test_X, test_Y)

print("Testing score: %.2f RMSE" % rmse_test)

Testing score: 27.65 RMSE


## 1000 Epochs

In [15]:
def fit_LSTM(train_X, train_Y, window = 1, neurons=128):
    set_random_seed(3)
    model = Sequential()
    
    model.add(LSTM(neurons, 
                   input_shape = (1, window)
                  ))
    model.add(Dense(1))
    model.compile(loss = "mean_squared_error", 
                  optimizer = "adam")
    earlyStop=EarlyStopping(monitor="val_loss",verbose=2,patience=15)
    model.fit(train_X, 
              train_Y, 
              epochs = 1000, 
              batch_size = 1,
              shuffle = False
             # verbose = 2
             )
    
    return(model)

# Fit the first model.
model1 = fit_LSTM(train_X, train_Y, window)

Epoch 1/1000
311/311 [==============================] - 4s 11ms/step - loss: 3.8197e-04
Epoch 2/1000
311/311 [==============================] - 2s 6ms/step - loss: 0.0025
Epoch 3/1000
311/311 [==============================] - 2s 6ms/step - loss: 0.0011
Epoch 4/1000
311/311 [==============================] - 2s 5ms/step - loss: 5.2735e-04
Epoch 5/1000
311/311 [==============================] - 2s 6ms/step - loss: 2.7239e-04
Epoch 6/1000
311/311 [==============================] - 2s 6ms/step - loss: 1.8980e-04
Epoch 7/1000
311/311 [==============================] - 2s 6ms/step - loss: 1.6834e-04
Epoch 8/1000
311/311 [==============================] - 2s 6ms/step - loss: 1.6428e-04
Epoch 9/1000
311/311 [==============================] - 2s 6ms/step - loss: 1.6537e-04
Epoch 10/1000
311/311 [==============================] - 2s 6ms/step - loss: 1.6809e-04
Epoch 11/1000
311/311 [==============================] - 2s 6ms/step - loss: 1.7119e-04
Epoch 12/1000
311/311 [=========================

Epoch 186/1000
311/311 [==============================] - 2s 6ms/step - loss: 1.4283e-04
Epoch 187/1000
311/311 [==============================] - 2s 6ms/step - loss: 1.4280e-04
Epoch 188/1000
311/311 [==============================] - 2s 6ms/step - loss: 1.4278e-04
Epoch 189/1000
311/311 [==============================] - 2s 6ms/step - loss: 1.4275e-04
Epoch 190/1000
311/311 [==============================] - 2s 6ms/step - loss: 1.4272e-04
Epoch 191/1000
311/311 [==============================] - 2s 6ms/step - loss: 1.4270e-04
Epoch 192/1000
311/311 [==============================] - 2s 7ms/step - loss: 1.4268e-04
Epoch 193/1000
311/311 [==============================] - 2s 7ms/step - loss: 1.4265e-04
Epoch 194/1000
311/311 [==============================] - 2s 6ms/step - loss: 1.4263e-04
Epoch 195/1000
311/311 [==============================] - 2s 7ms/step - loss: 1.4261e-04
Epoch 196/1000
311/311 [==============================] - 2s 7ms/step - loss: 1.4258e-04
Epoch 197/1000
311/31

311/311 [==============================] - 2s 6ms/step - loss: 1.3959e-04
Epoch 370/1000
311/311 [==============================] - 2s 6ms/step - loss: 1.3958e-04
Epoch 371/1000
311/311 [==============================] - 2s 6ms/step - loss: 1.3957e-04
Epoch 372/1000
311/311 [==============================] - 2s 6ms/step - loss: 1.3954e-04
Epoch 373/1000
311/311 [==============================] - 2s 6ms/step - loss: 1.3952e-04
Epoch 374/1000
311/311 [==============================] - 2s 6ms/step - loss: 1.3952e-04
Epoch 375/1000
311/311 [==============================] - 2s 6ms/step - loss: 1.3950e-04
Epoch 376/1000
311/311 [==============================] - 2s 6ms/step - loss: 1.3948e-04
Epoch 377/1000
311/311 [==============================] - 2s 6ms/step - loss: 1.3947e-04
Epoch 378/1000
311/311 [==============================] - 2s 6ms/step - loss: 1.3945e-04
Epoch 379/1000
311/311 [==============================] - 2s 6ms/step - loss: 1.3943e-04
Epoch 380/1000
311/311 [============

311/311 [==============================] - 2s 6ms/step - loss: 1.3746e-04
Epoch 554/1000
311/311 [==============================] - 2s 6ms/step - loss: 1.3746e-04
Epoch 555/1000
311/311 [==============================] - 2s 7ms/step - loss: 1.3745e-04
Epoch 556/1000
311/311 [==============================] - 2s 6ms/step - loss: 1.3744e-04
Epoch 557/1000
311/311 [==============================] - 2s 6ms/step - loss: 1.3743e-04
Epoch 558/1000
311/311 [==============================] - 2s 6ms/step - loss: 1.3742e-04
Epoch 559/1000
311/311 [==============================] - 2s 7ms/step - loss: 1.3741e-04
Epoch 560/1000
311/311 [==============================] - 2s 7ms/step - loss: 1.3740e-04
Epoch 561/1000
311/311 [==============================] - 2s 6ms/step - loss: 1.3739e-04
Epoch 562/1000
311/311 [==============================] - 2s 6ms/step - loss: 1.3739e-04
Epoch 563/1000
311/311 [==============================] - 2s 6ms/step - loss: 1.3738e-04
Epoch 564/1000
311/311 [============

311/311 [==============================] - 2s 6ms/step - loss: 1.3621e-04
Epoch 738/1000
311/311 [==============================] - 2s 7ms/step - loss: 1.3621e-04
Epoch 739/1000
311/311 [==============================] - 2s 6ms/step - loss: 1.3620e-04
Epoch 740/1000
311/311 [==============================] - 2s 6ms/step - loss: 1.3619e-04
Epoch 741/1000
311/311 [==============================] - 2s 7ms/step - loss: 1.3619e-04
Epoch 742/1000
311/311 [==============================] - 2s 7ms/step - loss: 1.3618e-04
Epoch 743/1000
311/311 [==============================] - 2s 6ms/step - loss: 1.3618e-04
Epoch 744/1000
311/311 [==============================] - 2s 6ms/step - loss: 1.3617e-04
Epoch 745/1000
311/311 [==============================] - 2s 6ms/step - loss: 1.3616e-04
Epoch 746/1000
311/311 [==============================] - 2s 6ms/step - loss: 1.3616e-04
Epoch 747/1000
311/311 [==============================] - 2s 6ms/step - loss: 1.3615e-04
Epoch 748/1000
311/311 [============

311/311 [==============================] - 2s 7ms/step - loss: 1.3567e-04
Epoch 830/1000
311/311 [==============================] - 2s 7ms/step - loss: 1.3566e-04
Epoch 831/1000
311/311 [==============================] - 2s 6ms/step - loss: 1.3565e-04
Epoch 832/1000
311/311 [==============================] - 2s 6ms/step - loss: 1.3565e-04
Epoch 833/1000
311/311 [==============================] - 2s 6ms/step - loss: 1.3564e-04
Epoch 834/1000
311/311 [==============================] - 2s 7ms/step - loss: 1.3564e-04
Epoch 835/1000
311/311 [==============================] - 2s 7ms/step - loss: 1.3563e-04
Epoch 836/1000
311/311 [==============================] - 2s 7ms/step - loss: 1.3562e-04
Epoch 837/1000
311/311 [==============================] - 2s 6ms/step - loss: 1.3562e-04
Epoch 838/1000
311/311 [==============================] - 2s 6ms/step - loss: 1.3561e-04
Epoch 839/1000
311/311 [==============================] - 2s 6ms/step - loss: 1.3561e-04
Epoch 840/1000
311/311 [============

In [16]:
rmse_test, test_pred = prediction_score(model1, test_X, test_Y)

print("Testing score: %.2f RMSE" % rmse_test)

Testing score: 11.88 RMSE


## 2000 Epochs

In [15]:
def fit_LSTM(train_X, train_Y, window = 1, neurons=128):
    set_random_seed(3)
    model = Sequential()
    
    model.add(LSTM(neurons, 
                   input_shape = (1, window)
                  ))
    model.add(Dense(1))
    model.compile(loss = "mean_squared_error", 
                  optimizer = "adam")
    earlyStop=EarlyStopping(monitor="val_loss",verbose=2,patience=15)
    model.fit(train_X, 
              train_Y, 
              epochs = 2000, 
              batch_size = 1,
              shuffle = False
             # verbose = 2
             )
    
    return(model)

# Fit the first model.
model1 = fit_LSTM(train_X, train_Y, window)

Epoch 1/2000
311/311 [==============================] - 2s 7ms/step - loss: 0.0633
Epoch 2/2000
311/311 [==============================] - 0s 282us/step - loss: 0.0371
Epoch 3/2000
311/311 [==============================] - 0s 278us/step - loss: 0.0213
Epoch 4/2000
311/311 [==============================] - 0s 288us/step - loss: 0.0141
Epoch 5/2000
311/311 [==============================] - 0s 279us/step - loss: 0.0127
Epoch 6/2000
311/311 [==============================] - 0s 281us/step - loss: 0.0131
Epoch 7/2000
311/311 [==============================] - 0s 275us/step - loss: 0.0131
Epoch 8/2000
311/311 [==============================] - 0s 276us/step - loss: 0.0125
Epoch 9/2000
311/311 [==============================] - 0s 287us/step - loss: 0.0117
Epoch 10/2000
311/311 [==============================] - 0s 294us/step - loss: 0.0110
Epoch 11/2000
311/311 [==============================] - 0s 280us/step - loss: 0.0102
Epoch 12/2000
311/311 [==============================] - 0s 288us

311/311 [==============================] - 0s 280us/step - loss: 1.2664e-04
Epoch 184/2000
311/311 [==============================] - 0s 274us/step - loss: 1.2662e-04
Epoch 185/2000
311/311 [==============================] - 0s 290us/step - loss: 1.2659e-04
Epoch 186/2000
311/311 [==============================] - 0s 287us/step - loss: 1.2657e-04
Epoch 187/2000
311/311 [==============================] - 0s 279us/step - loss: 1.2655e-04
Epoch 188/2000
311/311 [==============================] - 0s 278us/step - loss: 1.2652e-04
Epoch 189/2000
311/311 [==============================] - 0s 276us/step - loss: 1.2650e-04
Epoch 190/2000
311/311 [==============================] - 0s 280us/step - loss: 1.2647e-04
Epoch 191/2000
311/311 [==============================] - 0s 279us/step - loss: 1.2645e-04
Epoch 192/2000
311/311 [==============================] - 0s 290us/step - loss: 1.2642e-04
Epoch 193/2000
311/311 [==============================] - 0s 290us/step - loss: 1.2640e-04
Epoch 194/2000

311/311 [==============================] - 0s 286us/step - loss: 1.2288e-04
Epoch 364/2000
311/311 [==============================] - 0s 278us/step - loss: 1.2288e-04
Epoch 365/2000
311/311 [==============================] - 0s 280us/step - loss: 1.2288e-04
Epoch 366/2000
311/311 [==============================] - 0s 280us/step - loss: 1.2288e-04
Epoch 367/2000
311/311 [==============================] - 0s 289us/step - loss: 1.2289e-04
Epoch 368/2000
311/311 [==============================] - 0s 283us/step - loss: 1.2289e-04
Epoch 369/2000
311/311 [==============================] - 0s 292us/step - loss: 1.2290e-04
Epoch 370/2000
311/311 [==============================] - 0s 297us/step - loss: 1.2290e-04
Epoch 371/2000
311/311 [==============================] - 0s 284us/step - loss: 1.2291e-04
Epoch 372/2000
311/311 [==============================] - 0s 275us/step - loss: 1.2291e-04
Epoch 373/2000
311/311 [==============================] - 0s 282us/step - loss: 1.2292e-04
Epoch 374/2000

311/311 [==============================] - 0s 297us/step - loss: 1.3354e-04
Epoch 544/2000
311/311 [==============================] - 0s 289us/step - loss: 1.6629e-04
Epoch 545/2000
311/311 [==============================] - 0s 284us/step - loss: 1.3527e-04
Epoch 546/2000
311/311 [==============================] - 0s 281us/step - loss: 1.7763e-04
Epoch 547/2000
311/311 [==============================] - 0s 280us/step - loss: 1.3269e-04
Epoch 548/2000
311/311 [==============================] - 0s 282us/step - loss: 2.1095e-04
Epoch 549/2000
311/311 [==============================] - 0s 281us/step - loss: 1.7252e-04
Epoch 550/2000
311/311 [==============================] - 0s 290us/step - loss: 2.1595e-04
Epoch 551/2000
311/311 [==============================] - 0s 285us/step - loss: 2.0666e-04
Epoch 552/2000
311/311 [==============================] - 0s 282us/step - loss: 2.2651e-04
Epoch 553/2000
311/311 [==============================] - 0s 289us/step - loss: 3.3723e-04
Epoch 554/2000

311/311 [==============================] - 0s 312us/step - loss: 2.0541e-04
Epoch 724/2000
311/311 [==============================] - 0s 304us/step - loss: 1.5425e-04
Epoch 725/2000
311/311 [==============================] - 0s 308us/step - loss: 2.8786e-04
Epoch 726/2000
311/311 [==============================] - 0s 300us/step - loss: 1.7664e-04
Epoch 727/2000
311/311 [==============================] - 0s 295us/step - loss: 3.4349e-04
Epoch 728/2000
311/311 [==============================] - 0s 294us/step - loss: 2.0201e-04
Epoch 729/2000
311/311 [==============================] - 0s 295us/step - loss: 4.9739e-04
Epoch 730/2000
311/311 [==============================] - 0s 307us/step - loss: 2.4970e-04
Epoch 731/2000
311/311 [==============================] - 0s 301us/step - loss: 5.5404e-04
Epoch 732/2000
311/311 [==============================] - 0s 297us/step - loss: 2.9357e-04
Epoch 733/2000
311/311 [==============================] - 0s 304us/step - loss: 6.8483e-04
Epoch 734/2000

...

311/311 [==============================] - 0s 304us/step - loss: 1.2982e-04
Epoch 1261/2000
311/311 [==============================] - 0s 297us/step - loss: 1.3790e-04
Epoch 1262/2000
311/311 [==============================] - 0s 295us/step - loss: 1.3372e-04
Epoch 1263/2000
311/311 [==============================] - 0s 299us/step - loss: 1.2827e-04
Epoch 1264/2000
311/311 [==============================] - 0s 292us/step - loss: 1.2707e-04
Epoch 1265/2000
311/311 [==============================] - 0s 287us/step - loss: 1.2362e-04
Epoch 1266/2000
311/311 [==============================] - 0s 288us/step - loss: 1.2770e-04
Epoch 1267/2000
311/311 [==============================] - 0s 287us/step - loss: 1.2240e-04
Epoch 1268/2000
311/311 [==============================] - 0s 289us/step - loss: 1.2275e-04
Epoch 1269/2000
311/311 [==============================] - 0s 288us/step - loss: 1.2439e-04
Epoch 1270/2000
311/311 [==============================] - 0s 286us/step - loss: 1.2271e-04
Epoc

311/311 [==============================] - 0s 308us/step - loss: 1.2286e-04
Epoch 1439/2000
311/311 [==============================] - 0s 313us/step - loss: 1.2291e-04
Epoch 1440/2000
311/311 [==============================] - 0s 309us/step - loss: 1.2304e-04
Epoch 1441/2000
311/311 [==============================] - 0s 310us/step - loss: 1.2718e-04
Epoch 1442/2000
311/311 [==============================] - 0s 304us/step - loss: 1.2342e-04
Epoch 1443/2000
311/311 [==============================] - 0s 307us/step - loss: 1.2419e-04
Epoch 1444/2000
311/311 [==============================] - 0s 309us/step - loss: 1.2800e-04
Epoch 1445/2000
311/311 [==============================] - 0s 300us/step - loss: 1.2919e-04
Epoch 1446/2000
311/311 [==============================] - 0s 304us/step - loss: 1.2864e-04
Epoch 1447/2000
311/311 [==============================] - 0s 313us/step - loss: 1.2591e-04
Epoch 1448/2000
311/311 [==============================] - 0s 304us/step - loss: 1.4214e-04
Epoc

311/311 [==============================] - 0s 303us/step - loss: 1.2854e-04
Epoch 1617/2000
311/311 [==============================] - 0s 307us/step - loss: 1.3680e-04
Epoch 1618/2000
311/311 [==============================] - 0s 298us/step - loss: 1.2338e-04
Epoch 1619/2000
311/311 [==============================] - 0s 306us/step - loss: 1.2507e-04
Epoch 1620/2000
311/311 [==============================] - 0s 306us/step - loss: 1.2580e-04
Epoch 1621/2000
311/311 [==============================] - 0s 305us/step - loss: 1.2304e-04
Epoch 1622/2000
311/311 [==============================] - 0s 299us/step - loss: 1.2276e-04
Epoch 1623/2000
311/311 [==============================] - 0s 305us/step - loss: 1.2252e-04
Epoch 1624/2000
311/311 [==============================] - 0s 318us/step - loss: 1.2335e-04
Epoch 1625/2000
311/311 [==============================] - 0s 332us/step - loss: 1.2170e-04
Epoch 1626/2000
311/311 [==============================] - 0s 314us/step - loss: 1.2174e-04
Epoc

311/311 [==============================] - 0s 302us/step - loss: 1.2145e-04
Epoch 1795/2000
311/311 [==============================] - 0s 295us/step - loss: 1.2166e-04
Epoch 1796/2000
311/311 [==============================] - 0s 298us/step - loss: 1.2171e-04
Epoch 1797/2000
311/311 [==============================] - 0s 299us/step - loss: 1.2147e-04
Epoch 1798/2000
311/311 [==============================] - 0s 304us/step - loss: 1.2154e-04
Epoch 1799/2000
311/311 [==============================] - 0s 310us/step - loss: 1.2183e-04
Epoch 1800/2000
311/311 [==============================] - 0s 304us/step - loss: 1.2158e-04
Epoch 1801/2000
311/311 [==============================] - 0s 304us/step - loss: 1.2143e-04
Epoch 1802/2000
311/311 [==============================] - 0s 305us/step - loss: 1.2179e-04
Epoch 1803/2000
311/311 [==============================] - 0s 298us/step - loss: 1.2185e-04
Epoch 1804/2000
311/311 [==============================] - 0s 293us/step - loss: 1.2145e-04
Epoc

311/311 [==============================] - 0s 300us/step - loss: 1.2162e-04
Epoch 1973/2000
311/311 [==============================] - 0s 288us/step - loss: 1.2152e-04
Epoch 1974/2000
311/311 [==============================] - 0s 292us/step - loss: 1.2145e-04
Epoch 1975/2000
311/311 [==============================] - 0s 290us/step - loss: 1.2162e-04
Epoch 1976/2000
311/311 [==============================] - 0s 296us/step - loss: 1.2163e-04
Epoch 1977/2000
311/311 [==============================] - 0s 332us/step - loss: 1.2144e-04
Epoch 1978/2000
311/311 [==============================] - 0s 314us/step - loss: 1.2153e-04
Epoch 1979/2000
311/311 [==============================] - 0s 314us/step - loss: 1.2177e-04
Epoch 1980/2000
311/311 [==============================] - 0s 313us/step - loss: 1.2152e-04
Epoch 1981/2000
311/311 [==============================] - 0s 322us/step - loss: 1.2141e-04
Epoch 1982/2000
311/311 [==============================] - 0s 319us/step - loss: 1.2180e-04
Epoc

In [16]:
rmse_test, test_pred = prediction_score(model1, test_X, test_Y)
print("Testing score: %.2f RMSE" % rmse_test)

Testing score: 4.47 RMSE


## 2500 Epochs

In [22]:
def fit_LSTM(train_X, train_Y, window = 1, neurons=128):
    set_random_seed(3)
    model = Sequential()
    
    model.add(LSTM(neurons, 
                   input_shape = (1, window)
                  ))
    model.add(Dense(1))
    model.compile(loss = "mean_squared_error", 
                  optimizer = "adam")
    earlyStop=EarlyStopping(monitor="val_loss",verbose=2,patience=15)
    model.fit(train_X, 
              train_Y, 
              epochs = 2500, 
              batch_size = 1,
              shuffle = False
             # verbose = 2
             )
    
    return(model)

# Fit the first model.
model1 = fit_LSTM(train_X, train_Y, window)

Epoch 1/3000
311/311 [==============================] - 4s 13ms/step - loss: 3.8763e-04
Epoch 2/3000
311/311 [==============================] - 2s 5ms/step - loss: 0.0026
Epoch 3/3000
311/311 [==============================] - 2s 6ms/step - loss: 0.0012
Epoch 4/3000
311/311 [==============================] - 2s 6ms/step - loss: 5.3426e-04
Epoch 5/3000
311/311 [==============================] - 2s 6ms/step - loss: 2.7702e-04
Epoch 6/3000
311/311 [==============================] - 2s 6ms/step - loss: 1.9194e-04
Epoch 7/3000
311/311 [==============================] - 2s 6ms/step - loss: 1.6928e-04
Epoch 8/3000
311/311 [==============================] - 2s 6ms/step - loss: 1.6478e-04
Epoch 9/3000
311/311 [==============================] - 2s 6ms/step - loss: 1.6573e-04
Epoch 10/3000
311/311 [==============================] - 2s 6ms/step - loss: 1.6844e-04
Epoch 11/3000
311/311 [==============================] - 2s 6ms/step - loss: 1.7161e-04
Epoch 12/3000
311/311 [=========================

Epoch 186/3000
311/311 [==============================] - 2s 6ms/step - loss: 1.4292e-04
Epoch 187/3000
311/311 [==============================] - 2s 6ms/step - loss: 1.4289e-04
Epoch 188/3000
311/311 [==============================] - 3s 9ms/step - loss: 1.4286e-04
Epoch 189/3000
311/311 [==============================] - 4s 12ms/step - loss: 1.4283e-04
Epoch 190/3000
311/311 [==============================] - 4s 12ms/step - loss: 1.4280e-04
Epoch 191/3000
311/311 [==============================] - 4s 11ms/step - loss: 1.4278e-04
Epoch 192/3000
311/311 [==============================] - 4s 11ms/step - loss: 1.4275e-04
Epoch 193/3000
311/311 [==============================] - 4s 11ms/step - loss: 1.4272e-04
Epoch 194/3000
311/311 [==============================] - 4s 11ms/step - loss: 1.4270e-04
Epoch 195/3000
311/311 [==============================] - 4s 12ms/step - loss: 1.4267e-04
Epoch 196/3000
311/311 [==============================] - 4s 12ms/step - loss: 1.4264e-04
Epoch 197/300

311/311 [==============================] - 4s 11ms/step - loss: 1.4086e-04
Epoch 278/3000
311/311 [==============================] - 4s 11ms/step - loss: 1.4084e-04
Epoch 279/3000
311/311 [==============================] - 4s 11ms/step - loss: 1.4082e-04
Epoch 280/3000
311/311 [==============================] - 4s 11ms/step - loss: 1.4080e-04
Epoch 281/3000
311/311 [==============================] - 4s 11ms/step - loss: 1.4077e-04
Epoch 282/3000
311/311 [==============================] - 4s 11ms/step - loss: 1.4076e-04
Epoch 283/3000
311/311 [==============================] - 4s 11ms/step - loss: 1.4073e-04
Epoch 284/3000
311/311 [==============================] - 4s 12ms/step - loss: 1.4071e-04
Epoch 285/3000
311/311 [==============================] - 4s 12ms/step - loss: 1.4069e-04
Epoch 286/3000
311/311 [==============================] - 4s 12ms/step - loss: 1.4067e-04
Epoch 287/3000
311/311 [==============================] - 4s 12ms/step - loss: 1.4065e-04
Epoch 288/3000
311/311 [=

311/311 [==============================] - 4s 14ms/step - loss: 1.3794e-04
Epoch 460/3000
311/311 [==============================] - 4s 14ms/step - loss: 1.3793e-04
Epoch 461/3000
311/311 [==============================] - 4s 14ms/step - loss: 1.3791e-04
Epoch 462/3000
311/311 [==============================] - 4s 14ms/step - loss: 1.3789e-04
Epoch 463/3000
311/311 [==============================] - 4s 14ms/step - loss: 1.3788e-04
Epoch 464/3000
311/311 [==============================] - 4s 14ms/step - loss: 1.3787e-04
Epoch 465/3000
311/311 [==============================] - 4s 14ms/step - loss: 1.3786e-04
Epoch 466/3000
311/311 [==============================] - 4s 14ms/step - loss: 1.3784e-04
Epoch 467/3000
311/311 [==============================] - 4s 14ms/step - loss: 1.3783e-04
Epoch 468/3000
311/311 [==============================] - 4s 14ms/step - loss: 1.3782e-04
Epoch 469/3000
311/311 [==============================] - 4s 14ms/step - loss: 1.3781e-04
Epoch 470/3000
311/311 [=

311/311 [==============================] - 3s 9ms/step - loss: 1.3689e-04
Epoch 552/3000
311/311 [==============================] - 3s 9ms/step - loss: 1.3688e-04
Epoch 553/3000
311/311 [==============================] - 3s 9ms/step - loss: 1.3688e-04
Epoch 554/3000
311/311 [==============================] - 3s 9ms/step - loss: 1.3687e-04
Epoch 555/3000
311/311 [==============================] - 3s 9ms/step - loss: 1.3686e-04
Epoch 556/3000
311/311 [==============================] - 3s 9ms/step - loss: 1.3685e-04
Epoch 557/3000
311/311 [==============================] - 3s 8ms/step - loss: 1.3684e-04
Epoch 558/3000
311/311 [==============================] - 3s 8ms/step - loss: 1.3683e-04
Epoch 559/3000
311/311 [==============================] - 3s 8ms/step - loss: 1.3682e-04
Epoch 560/3000
311/311 [==============================] - 3s 8ms/step - loss: 1.3682e-04
Epoch 561/3000
311/311 [==============================] - 3s 9ms/step - loss: 1.3681e-04
Epoch 562/3000
311/311 [============

...

311/311 [==============================] - 3s 8ms/step - loss: 1.3059e-04
Epoch 2193/3000
311/311 [==============================] - 3s 8ms/step - loss: 1.3058e-04
Epoch 2194/3000
311/311 [==============================] - 3s 8ms/step - loss: 1.3058e-04
Epoch 2195/3000
311/311 [==============================] - 3s 8ms/step - loss: 1.3058e-04
Epoch 2196/3000
311/311 [==============================] - 3s 8ms/step - loss: 1.3058e-04
Epoch 2197/3000
311/311 [==============================] - 3s 8ms/step - loss: 1.3058e-04
Epoch 2198/3000
311/311 [==============================] - 3s 8ms/step - loss: 1.3058e-04
Epoch 2199/3000
311/311 [==============================] - 3s 9ms/step - loss: 1.3058e-04
Epoch 2200/3000
311/311 [==============================] - 3s 8ms/step - loss: 1.3057e-04
Epoch 2201/3000
311/311 [==============================] - 3s 8ms/step - loss: 1.3057e-04
Epoch 2202/3000
311/311 [==============================] - 3s 8ms/step - loss: 1.3057e-04
Epoch 2203/3000
311/311 [=

311/311 [==============================] - 3s 8ms/step - loss: 1.3038e-04
Epoch 2375/3000
311/311 [==============================] - 3s 8ms/step - loss: 1.3038e-04
Epoch 2376/3000
311/311 [==============================] - 2s 8ms/step - loss: 1.3038e-04
Epoch 2377/3000
311/311 [==============================] - 3s 8ms/step - loss: 1.3036e-04
Epoch 2378/3000
311/311 [==============================] - 3s 8ms/step - loss: 1.3036e-04
Epoch 2379/3000
311/311 [==============================] - 3s 8ms/step - loss: 1.3037e-04
Epoch 2380/3000
311/311 [==============================] - 3s 8ms/step - loss: 1.3037e-04
Epoch 2381/3000
311/311 [==============================] - 3s 8ms/step - loss: 1.3038e-04
Epoch 2382/3000
311/311 [==============================] - 2s 8ms/step - loss: 1.3038e-04
Epoch 2383/3000
311/311 [==============================] - 3s 8ms/step - loss: 1.3036e-04
Epoch 2384/3000
311/311 [==============================] - 2s 8ms/step - loss: 1.3036e-04
Epoch 2385/3000
311/311 [=

311/311 [==============================] - 3s 8ms/step - loss: 1.3023e-04
Epoch 2556/3000
311/311 [==============================] - 3s 8ms/step - loss: 1.3022e-04
Epoch 2557/3000
311/311 [==============================] - 3s 8ms/step - loss: 1.3022e-04
Epoch 2558/3000
311/311 [==============================] - 3s 8ms/step - loss: 1.3024e-04
Epoch 2559/3000
311/311 [==============================] - 2s 8ms/step - loss: 1.3022e-04
Epoch 2560/3000
311/311 [==============================] - 2s 8ms/step - loss: 1.3023e-04
Epoch 2561/3000
311/311 [==============================] - 3s 8ms/step - loss: 1.3021e-04
Epoch 2562/3000
311/311 [==============================] - 2s 8ms/step - loss: 1.3022e-04
Epoch 2563/3000
311/311 [==============================] - 3s 8ms/step - loss: 1.3021e-04
Epoch 2564/3000
311/311 [==============================] - 2s 8ms/step - loss: 1.3022e-04
Epoch 2565/3000
311/311 [==============================] - 2s 8ms/step - loss: 1.3023e-04
Epoch 2566/3000
311/311 [=

Epoch 2736/3000
311/311 [==============================] - 3s 9ms/step - loss: 1.3013e-04
Epoch 2737/3000
311/311 [==============================] - 3s 9ms/step - loss: 1.3014e-04
Epoch 2738/3000
311/311 [==============================] - 3s 9ms/step - loss: 1.3013e-04
Epoch 2739/3000
311/311 [==============================] - 3s 8ms/step - loss: 1.3014e-04
Epoch 2740/3000
311/311 [==============================] - 2s 8ms/step - loss: 1.3014e-04
Epoch 2741/3000
311/311 [==============================] - 2s 8ms/step - loss: 1.3013e-04
Epoch 2742/3000
311/311 [==============================] - 3s 8ms/step - loss: 1.3014e-04
Epoch 2743/3000
311/311 [==============================] - 3s 9ms/step - loss: 1.3013e-04
Epoch 2744/3000
311/311 [==============================] - 3s 8ms/step - loss: 1.3014e-04
Epoch 2745/3000
311/311 [==============================] - 3s 8ms/step - loss: 1.3014e-04
Epoch 2746/3000
311/311 [==============================] - 3s 8ms/step - loss: 1.3013e-04
Epoch 2747

311/311 [==============================] - 3s 9ms/step - loss: 1.3012e-04
Epoch 2828/3000
311/311 [==============================] - 3s 9ms/step - loss: 1.3011e-04
Epoch 2829/3000
311/311 [==============================] - 3s 8ms/step - loss: 1.3012e-04
Epoch 2830/3000
311/311 [==============================] - 3s 9ms/step - loss: 1.3012e-04
Epoch 2831/3000
311/311 [==============================] - 3s 8ms/step - loss: 1.3011e-04
Epoch 2832/3000
311/311 [==============================] - 3s 8ms/step - loss: 1.3012e-04
Epoch 2833/3000
311/311 [==============================] - 3s 8ms/step - loss: 1.3012e-04
Epoch 2834/3000
311/311 [==============================] - 3s 9ms/step - loss: 1.3011e-04
Epoch 2835/3000
311/311 [==============================] - 3s 8ms/step - loss: 1.3010e-04
Epoch 2836/3000
311/311 [==============================] - 3s 8ms/step - loss: 1.3012e-04
Epoch 2837/3000
311/311 [==============================] - 3s 9ms/step - loss: 1.3012e-04
Epoch 2838/3000
311/311 [=

In [23]:
rmse_test, test_pred = prediction_score(model1, test_X, test_Y)
print("Testing score: %.2f RMSE" % rmse_test)

Testing score: 16.81 RMSE
